<a href="https://colab.research.google.com/github/robinjameslee/SHA256-in-Python/blob/main/SHA256_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What is SHA256?

SHA-256 is a part of SHA-2 (Secure Hash Algorithm 2), and is one of the most popular hash algorithms, likely because of Cryptocurrency's adoption of the mehtod. 

In this Notebook, I'll try to calculate the SHA256 result of a string step by step.

Reference:
*   https://www.movable-type.co.uk/scripts/sha256.html
*   https://blog.boot.dev/cryptography/how-sha-2-works-step-by-step-sha-256/

In [47]:
import hashlib
import math

First let's use python's hashlib package to calculate the hashed string

In [48]:
sample_str = 'SHA256 in Python'

hased_str = hashlib.sha256(sample_str.encode())
print(hased_str.hexdigest())

3a2375f5162bf259ad04ce7b92425e86698768eb8dab4ae00660d5cf02458770


Let's try to calculate the above result step by step without using the package

The main purposes of a hash function are:
1. To randomize the data
2. To accept an input of vary length and return a fixed-length output
3. To make sure the input couldn't be derived from the output

In [49]:
# 1. Convert our sample string to binary
sample_str_binary = ''.join(f'{ord(i):08b}' for i in sample_str)

# 2. Append a '1'
sample_str_binary += '1'

# 3. Append '0's to make the length of the binaries to a multiple of 512 less 64 bits
len_sample_str_binary = len(sample_str_binary)
multiple_of_512 = math.ceil(len_sample_str_binary / 512)

padded_bits = multiple_of_512 * 512 - len_sample_str_binary - 8
sample_str_binary += '0' * padded_bits

# 4. Append 64 bits to the end where they're the binary form of the length of the our sample string
sample_str_binary += format(len(sample_str) * 8, '08b')
print(f'length: {len(sample_str_binary)}, {sample_str_binary}')

length: 512, 01010011010010000100000100110010001101010011011000100000011010010110111000100000010100000111100101110100011010000110111101101110100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000


In [50]:
# 5. Initialize Hash Values (h)
# Here we create 8 hash values which are the first 32 bits of the fractional parts of the square root of the first 8 prime numbers 2, 3, 5, 7, 11, 13, 17, 19
def initialize_hash_values(n):
  res = math.sqrt(n)
  res = math.modf(res)[0] # take the decimals only
  # res = hex(int(res * (2**32))) # backup function if you'd like the number in hex format
  res = int(res * (2**32))
  return f'{res:032b}' # return in 32 bits

def hex_to_32bits(n):
  return f'{int(n, 16):032b}'

h0 = initialize_hash_values(2) # h0 = 6A09E667 = 01101010000010011110011001100111
h1 = initialize_hash_values(3) # h1 = BB67AE85 = 10111011011001111010111010000101
h2 = initialize_hash_values(5) # h2 = 3C6EF372 = 00111100011011101111001101110010
h3 = initialize_hash_values(7) # h3 = A54FF53A = 10100101010011111111010100111010
h4 = initialize_hash_values(11) # h4 = 510E527F = 01010001000011100101001001111111
h5 = initialize_hash_values(13) # h5 = 9B05688C = 10011011000001010110100010001100
h6 = initialize_hash_values(17) # h6 = 1F83D9AB = 00011111100000111101100110101011
h7 = initialize_hash_values(19) # h7 = 5BE0CD19 = 01011011111000001100110100011001

# Initialize variables a, b, c, d, e, f, g, h and set them equal to the current hash values respectively. h0, h1, h2, h3, h4, h5, h6, h7
a = h0
b = h1
c = h2
d = h3
e = h4
f = h5
g = h6
h = h7

In [51]:
# 6. Initialize Round Constants (k)
# These are words representing the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers, in hex format
k = [
  '0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5',
  '0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174',
  '0xe49b69c1', '0xefbe4786', '0x0fc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da',
  '0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x06ca6351', '0x14292967',
  '0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85',
  '0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070',
  '0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3',
  '0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2'
]

K = [hex_to_32bits(i) for i in k] # convert them in 32 bits

In [52]:
# 7. Create Message Schedule (w)
# Copy the input data from step 1 into a new array where each entry is a 32-bit word:
sample_str_32bit = [sample_str_binary[i:i+32] for i in range(0, len(sample_str_binary), 32)]
sample_str_32bit

['01010011010010000100000100110010',
 '00110101001101100010000001101001',
 '01101110001000000101000001111001',
 '01110100011010000110111101101110',
 '10000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000010000000']

In [53]:
# 8. Add 48 more words initialized to zero, such that we have an array w[0…63]
sample_str_32bit += ['00000000000000000000000000000000'] * 48

In [54]:
def r_rotate(str, n): # right rotate
    return str[-n:] + str[:-n]

def r_shift(str, n): # right shift
    return n * '0' + str[:-n]

def bitwise_orp(symbol, *args):
    res = int(args[0], 2)
    for i in args[1:]:
      if symbol == '&':
        res &= int(i, 2)
      elif symbol == '|':
        res |= int(i, 2)
      elif symbol == '^':
        res ^= int(i, 2)
    return f'{res:032b}'

def bitwise_not(n):
    res = ~int(n, 2)
    return f'{res:032b}'

def add_mod32(*args):
    res = int(args[0], 2)
    for i in args[1:]:
        res += int(i, 2)
    res %= 2 ** 32
    return f'{res:032b}'

w = sample_str_32bit # take a copy

for i in range(16, 64):
    sigma_0 = bitwise_orp('^', r_rotate(w[i-15], 7), r_rotate(w[i-15], 18), r_shift(w[i-15], 3))
    sigma_1 = bitwise_orp('^', r_rotate(w[i-2], 17), r_rotate(w[i-2], 19), r_shift(w[i-2], 10))
    w[i] = add_mod32(w[i-16], sigma_0, w[i-7], sigma_1)

In [55]:
# 9. Run the compression loop as follows:
for i in range(64):
    sigma_1 = bitwise_orp('^', r_rotate(e, 6), r_rotate(e, 11), r_rotate(e, 25))
    choose = bitwise_orp('^', bitwise_orp('&', e, f), bitwise_orp('&', bitwise_not(e), g))
    temp_1 = add_mod32(h, sigma_1, choose, K[i], w[i])

    sigma_0 = bitwise_orp('^', r_rotate(a, 2), r_rotate(a, 13), r_rotate(a, 22))
    majority = bitwise_orp('^', bitwise_orp('&', a, b), bitwise_orp('&', a, c), bitwise_orp('&', b, c))
    temp_2 = add_mod32(sigma_0, majority)

    h = g
    g = f
    f = e
    e = add_mod32(d, temp_1)
    d = c
    c = b
    b = a
    a = add_mod32(temp_1, temp_2)

In [56]:
# 10. Modify Final Values
# Modify the hash values by adding their respective variables to them, a-h, in modulo 2^32.
h0 = add_mod32(h0, a)
h1 = add_mod32(h1, b)
h2 = add_mod32(h2, c)
h3 = add_mod32(h3, d)
h4 = add_mod32(h4, e)
h5 = add_mod32(h5, f)
h6 = add_mod32(h6, g)
h7 = add_mod32(h7, h)

In [57]:
# 11. Concatenate h0 to h7 to get our final output
final_output = ''.join([hex(int(i, 2))[2:] for i in [h0,h1,h2,h3,h4,h5,h6,h7]])
print(f'Our result: {final_output}')
print(f'Python package result: {hased_str.hexdigest()}')

Our result: 3a2375f5162bf259ad04ce7b92425e86698768eb8dab4ae0660d5cf2458770
Python package result: 3a2375f5162bf259ad04ce7b92425e86698768eb8dab4ae00660d5cf02458770
